# Create Neo4j Graph Database

First run notebook [1a-Strains.ipynb](./1a-Strains.ipynb) to create the node and relationship files in the /data directory.

[Neo4j Import Guide](https://neo4j.com/developer/guide-import-csv/)

[Neo4j Batch Import](https://neo4j.com/docs/operations-manual/current/tools/import/)

In [2]:
import os

In [3]:
NEO4J_HOME = os.getenv('NEO4J_HOME')
print(NEO4J_HOME)

C:\Users\ABM589\.Neo4jDesktop\neo4jDatabases\database-0792204b-06f5-45c8-a473-faab7638e73a\installation-4.0.1


In [3]:
# Print neo4j-admin help
# !"$NEO4J_HOME"/bin/neo4j-admin import

Remove existing database (can only batch upload into an empty database)

In [4]:
!rm -r "$NEO4J_HOME"/data/databases/graph.db

Batch upload nodes and relationships

In [19]:

nodes_files = []
relationships_files = []
for root, dirs, files in os.walk('../reference_data/nodes/'):
    for file in files: 
        nodes_files.append(os.path.join(root, file))
for root, dirs, files in os.walk('../data/nodes/'):
    for file in files: 
        nodes_files.append(os.path.join(root, file))
for root, dirs, files in os.walk('../reference_data/relationships/'):
    for file in files: 
        relationships_files.append(os.path.join(root, file))
for root, dirs, files in os.walk('../data/relationships/'):
    for file in files: 
        relationships_files.append(os.path.join(root, file))

command_tokens = ['!'+NEO4J_HOME+'/bin/neo4j-admin']
command_tokens.append('import')
command_tokens.append('--database=graph.db')
command_tokens.append('--id-type=STRING')
command_tokens.append('--max-memory=250000') 
for node in nodes_files:
    command_tokens.append('--nodes=' + node)
for relationship in relationships_files:
    command_tokens.append('--relationships=' + relationship)
    
print (command_tokens)

['!C:\\Users\\ABM589\\.Neo4jDesktop\\neo4jDatabases\\database-0792204b-06f5-45c8-a473-faab7638e73a\\installation-4.0.1/bin/neo4j-admin', 'import', '--database=graph.db', '--id-type=STRING', '--max-memory=250000', '--nodes=../reference_data/nodes/Dashboard.csv', '--nodes=../reference_data/nodes/Outbreak.csv', '--nodes=../reference_data/nodes/Pathogen.csv', '--nodes=../reference_data/nodes/ReferenceGenome.csv', '--nodes=../reference_data/nodes/.ipynb_checkpoints\\ReferenceGenome-checkpoint.csv', '--nodes=../data/nodes/Admin1.csv', '--nodes=../data/nodes/Admin2.csv', '--nodes=../data/nodes/City.csv', '--nodes=../data/nodes/Country.csv', '--nodes=../data/nodes/Host.csv', '--nodes=../data/nodes/PersonAnimal.csv', '--nodes=../data/nodes/Strain.csv', '--relationships=../reference_data/relationships/City-EXPLORE_IN-Dashboard.csv', '--relationships=../reference_data/relationships/Outbreak-EXPLORE_IN-Dashboard.csv', '--relationships=../reference_data/relationships/Pathogen-CAUSES-Outbreak.csv', 

In [20]:
import subprocess

subprocess.call(command_tokens)

FileNotFoundError: [WinError 2] The system cannot find the file specified